In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import re
from nltk.corpus import stopwords

In [2]:
with open('knowledge2.json', 'r') as file:
    data = json.load(file)

df = pd.DataFrame(data)
df.head()

,text_input,output
0,Dimana saja tempat belajar di BINUS University?,Tempat belajar di BINUS University meliputi pe...
1,Apa itu LKC (Library and Knowledge Center)?,LKC (Library and Knowledge Center) adalah perp...
2,Dimana lokasi LKC?,"LKC terletak di lantai 1, di depan main lobby."
3,Apa saja fasilitas yang tersedia di LKC?,LKC menyediakan fasilitas seperti ruang diskus...
4,Apa itu Beehub?,Beehub adalah tempat belajar yang terletak di ...


In [3]:
df.shape

(147, 2)

In [4]:
nltk.download('stopwords')

stop_words = set(stopwords.words('indonesian'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def remove_stopwords(text):
    # Regex untuk mendeteksi teks di antara tag <a> dan </a>
    anchor_tags = re.findall(r'<a.*?>.*?</a>', text)

    # Menghilangkan teks di antara tag <a> dan </a> dari teks utama
    text_without_anchors = re.sub(r'<a.*?>.*?</a>', ' <ANCHOR> ', text)

    # Menghapus stopwords dari teks utama yang sudah dihilangkan tag <a>
    words = text_without_anchors.split()
    words = [word for word in words if word.lower() not in stop_words]
    text_without_stopwords = ' '.join(words)

    # Mengembalikan teks di antara tag <a> ke tempat semula
    for anchor in anchor_tags:
        text_without_stopwords = text_without_stopwords.replace('<ANCHOR>', anchor, 1)

    return text_without_stopwords

In [6]:
df['text_input_clean'] = df['text_input'].apply(remove_stopwords)
df['output_clean'] = df['output'].apply(remove_stopwords)

In [7]:
df_result = df[['text_input_clean', 'output_clean']]

df_result.head()

,text_input_clean,output_clean
0,Dimana belajar BINUS University?,belajar BINUS University meliputi perpustakaan...
1,LKC (Library and Knowledge Center)?,LKC (Library and Knowledge Center) perpustakaa...
2,Dimana lokasi LKC?,"LKC terletak lantai 1, main lobby."
3,fasilitas tersedia LKC?,"LKC menyediakan fasilitas ruang diskusi, sofa ..."
4,Beehub?,Beehub belajar terletak gedung parkir kampus A...


# Convert df_result DataFrame to JSON format

In [8]:
output_json = df_result.to_json(orient='records')
output_json = output_json.replace('text_input_clean', 'text_input').replace('output_clean', 'output')


output_json = df_result.to_json(orient='records', indent=4)

with open('knowledge_clean.json', 'w') as file:
    file.write(output_json)

# Check knowledge_clean.json file

In [9]:
with open('knowledge_clean.json', 'r') as file:
    data = json.load(file)

df_clean = pd.DataFrame(data)
df_clean.head()

,text_input_clean,output_clean
0,Dimana belajar BINUS University?,belajar BINUS University meliputi perpustakaan...
1,LKC (Library and Knowledge Center)?,LKC (Library and Knowledge Center) perpustakaa...
2,Dimana lokasi LKC?,"LKC terletak lantai 1, main lobby."
3,fasilitas tersedia LKC?,"LKC menyediakan fasilitas ruang diskusi, sofa ..."
4,Beehub?,Beehub belajar terletak gedung parkir kampus A...
